In [1]:
%pip install tvdb_v4_official -q
%pip install python-dotenv -q
%pip install transformers datasets torch scikit-learn -q
%pip install spacy -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = "1e31ec95-4457-4cdc-b2d4-8ff602c85e07"
assert api_key, "No API key found. Please set the TVDB_API_KEY environment variable."

In [3]:
import pickle
from pathlib import Path
import dis

def p(name, func, log=False, cache='cache'):
    if cache[-1] == '/': cache = cache[:-1]
    
    Path(cache).mkdir(exist_ok=True)

    try:
        with open(f'{cache}/{name}.pkl', 'rb') as f:
            if log: print(f'success loading {cache}/{name}.pkl')
            return pickle.load(f)
    except:
        if log: print(f'failed loading {cache}/{name}.pkl')
        result = func()
        with open(f'{cache}/{name}.pkl', 'wb') as f:
            pickle.dump(result, f)
        if log: print(f'cache saved.')
        return result

In [4]:
import tvdb_v4_official
import itertools

tvdb = tvdb_v4_official.TVDB(api_key)
pokeid = 76703

pokemon = p('series', lambda: tvdb.get_series(pokeid))
languages = pokemon['overviewTranslations']
print('languages:', languages)

def get_eps(lang=None):
    eps_jp = []
    page = 0
    while True:
        eps = p(f'episodes_{lang or "jpn"}_{page}', lambda: tvdb.get_series_episodes(pokeid, page=page, lang=lang))['episodes']
        if not eps: break
        eps_jp += eps
        page += 1
    return eps_jp

eps_jpn = p('episodes_jpn', lambda: get_eps('jpn'))
eps_eng = p('episodes_eng', lambda: get_eps('eng'))
eps_fra = p('episodes_fra', lambda: get_eps('fra'))
eps_deu = p('episodes_deu', lambda: get_eps('deu'))
eps_spa = p('episodes_spa', lambda: get_eps('spa'))

languages: ['ces', 'deu', 'eng', 'fin', 'fra', 'heb', 'hun', 'ita', 'jpn', 'nld', 'por', 'rus', 'spa', 'swe', 'zho', 'nor', 'pol', 'pt', 'ara', 'zhtw', 'yue']


In [5]:
def count_overview(eps):
    return sum(1 for ep in eps if ep['overview'])

print('jpn:', count_overview(eps_jpn))
print('eng:', count_overview(eps_eng))
print('fra:', count_overview(eps_fra))
print('deu:', count_overview(eps_deu))
print('spa:', count_overview(eps_spa))


jpn: 462
eng: 1297
fra: 416
deu: 1260
spa: 1157


In [6]:
# only save the ones with overview in a df for each language

import pandas as pd
import numpy as np

def get_df(eps, lang):
    df = pd.DataFrame(eps)
    df['lang'] = lang
    df = df[df['overview'].notna()]
    return df

# df_jpn = get_df(eps_jpn, 'jpn')
df_eng = get_df(eps_eng, 'eng')
# df_fra = get_df(eps_fra, 'fra')
# df_deu = get_df(eps_deu, 'deu')
# df_spa = get_df(eps_spa, 'spa')


In [7]:
# only keep id and overview for each language and merge them into one df
df_eng2 = df_eng[['id', 'overview']]
df_eng2 = df_eng2.rename(columns={'overview': 'overview_eng'})

# df_fra2 = df_fra[['id', 'overview']]
# df_fra2 = df_fra2.rename(columns={'overview': 'overview_fra'})

# df_deu2 = df_deu[['id', 'overview']]
# df_deu2 = df_deu2.rename(columns={'overview': 'overview_deu'})

# df_spa2 = df_spa[['id', 'overview']]
# df_spa2 = df_spa2.rename(columns={'overview': 'overview_spa'})

# # merge the 4 dfs with ids
# df = df_fra2.merge(df_eng2, on='id', how='left')
# df = df.merge(df_deu2, on='id', how='left')
# df = df.merge(df_spa2, on='id', how='left')

# remove NAN rows
# df = df.replace(np.nan, '', regex=True)
# df = df[df['overview_eng'] != '']
# df = df[df['overview_fra'] != '']
# df = df[df['overview_deu'] != '']
# df = df[df['overview_spa'] != '']

#remove NAN from eng
df_eng2 = df_eng2.replace(np.nan, '', regex=True)
df = df_eng2
dfclone = df.copy()

df

,id,overview_eng
0,316019,"Scientists genetically create a new Pokémon, M..."
1,316020,Ash Ketchum must gather the three spheres of f...
2,208475,"In the town of Greenfield, a young, lonely gir..."
3,316023,"Mewtwo, his heart having been softened by the ..."
4,394456,There are always those who seek to contain and...
...,...,...
1335,10180470,Liko's grandmother has decided it's time to le...
1336,10186547,"As they return to Paldea, the Rising Volt Tack..."
1337,10239398,"During one of her livestreams, Dot has been as..."
1338,10239399,The Rising Volt Tacklers have answered a reque...


In [8]:
# df = df[df['overview_eng'] != df['overview_fra']]
# df = df[df['overview_eng'] != df['overview_deu']]
# df = df[df['overview_eng'] != df['overview_spa']]
# df = df[df['overview_fra'] != df['overview_deu']]
# df = df[df['overview_fra'] != df['overview_spa']]
# df = df[df['overview_deu'] != df['overview_spa']]
# df

In [9]:
overviews= df['overview_eng'].tolist()
len(overviews)

1297

In [10]:

entity_categories = {
    "POKEMON": ['Bulbasaur', 'Ivysaur', 'Venusaur', 'Venusaur Mega Venusaur', 'Charmander', 'Charmeleon', 'Charizard',
     'Charizard Mega Charizard X', 'Charizard Mega Charizard Y', 'Squirtle', 'Wartortle', 'Blastoise',
       'Blastoise Mega Blastoise', 'Caterpie', 'Metapod', 'Butterfree', 'Weedle', 'Kakuna', 'Beedrill',
         'Beedrill Mega Beedrill', 'Pidgey', 'Pidgeotto', 'Pidgeot', 'Pidgeot Mega Pidgeot', 'Rattata',
           'Rattata Alolan Rattata', 'Raticate', 'Raticate Alolan Raticate', 'Spearow', 'Fearow', 'Ekans',
             'Arbok', 'Pikachu', 'Pikachu Partner Pikachu', 'Raichu', 'Raichu Alolan Raichu', 'Sandshrew', 'Sandshrew Alolan Sandshrew', 'Sandslash', 'Sandslash Alolan Sandslash', 'Nidoran♀', 'Nidorina', 'Nidoqueen', 'Nidoran♂', 'Nidorino', 'Nidoking', 'Clefairy', 'Clefable', 'Vulpix', 'Vulpix Alolan Vulpix', 'Ninetales', 'Ninetales Alolan Ninetales', 'Jigglypuff', 'Wigglytuff', 'Zubat', 'Golbat', 'Oddish', 'Gloom', 'Vileplume', 'Paras', 'Parasect', 'Venonat', 'Venomoth', 'Diglett', 'Diglett Alolan Diglett', 'Dugtrio', 'Dugtrio Alolan Dugtrio', 'Meowth', 'Meowth Alolan Meowth', 'Meowth Galarian Meowth', 'Persian', 'Persian Alolan Persian', 'Psyduck', 'Golduck', 'Mankey', 'Primeape', 'Growlithe', 'Growlithe Hisuian Growlithe', 'Arcanine', 'Arcanine Hisuian Arcanine', 'Poliwag', 'Poliwhirl', 'Poliwrath', 'Abra', 'Kadabra', 'Alakazam', 'Alakazam Mega Alakazam', 'Machop', 'Machoke', 'Machamp', 'Bellsprout', 'Weepinbell', 'Victreebel', 'Tentacool', 'Tentacruel', 'Geodude', 'Geodude Alolan Geodude', 'Graveler', 'Graveler Alolan Graveler', 'Golem', 'Golem Alolan Golem', 'Ponyta', 'Ponyta Galarian Ponyta', 'Rapidash', 'Rapidash Galarian Rapidash', 'Slowpoke', 'Slowpoke Galarian Slowpoke', 'Slowbro', 'Slowbro Mega Slowbro', 'Slowbro Galarian Slowbro', 'Magnemite', 'Magneton', "Farfetch'd", "Farfetch'd Galarian Farfetch'd", 'Doduo', 'Dodrio', 'Seel', 'Dewgong', 'Grimer', 'Grimer Alolan Grimer', 'Muk', 'Muk Alolan Muk', 'Shellder', 'Cloyster', 'Gastly', 'Haunter', 'Gengar', 'Gengar Mega Gengar', 'Onix', 'Drowzee', 'Hypno', 'Krabby', 'Kingler', 'Voltorb', 'Voltorb Hisuian Voltorb', 'Electrode', 'Electrode Hisuian Electrode', 'Exeggcute', 'Exeggutor', 'Exeggutor Alolan Exeggutor', 'Cubone', 'Marowak', 'Marowak Alolan Marowak', 'Hitmonlee', 'Hitmonchan', 'Lickitung', 'Koffing', 'Weezing', 'Weezing Galarian Weezing', 'Rhyhorn', 'Rhydon', 'Chansey', 'Tangela', 'Kangaskhan', 'Kangaskhan Mega Kangaskhan', 'Horsea', 'Seadra', 'Goldeen', 'Seaking', 'Staryu', 'Starmie', 'Mr. Mime', 'Mr. Mime Galarian Mr. Mime', 'Scyther', 'Jynx', 'Electabuzz', 'Magmar', 'Pinsir', 'Pinsir Mega Pinsir', 'Tauros', 'Tauros Combat Breed', 'Tauros Blaze Breed', 'Tauros Aqua Breed', 'Magikarp', 'Gyarados', 'Gyarados Mega Gyarados', 'Lapras', 'Ditto', 'Eevee', 'Eevee Partner Eevee', 'Vaporeon', 'Jolteon', 'Flareon', 'Porygon', 'Omanyte', 'Omastar', 'Kabuto', 'Kabutops', 'Aerodactyl', 'Aerodactyl Mega Aerodactyl', 'Snorlax', 'Articuno', 'Articuno Galarian Articuno', 'Zapdos', 'Zapdos Galarian Zapdos', 'Moltres', 'Moltres Galarian Moltres', 'Dratini', 'Dragonair', 'Dragonite', 'Mewtwo', 'Mewtwo Mega Mewtwo X', 'Mewtwo Mega Mewtwo Y', 'Mew', 'Chikorita', 'Bayleef', 'Meganium', 'Cyndaquil', 'Quilava', 'Typhlosion', 'Typhlosion Hisuian Typhlosion', 'Totodile', 'Croconaw', 'Feraligatr', 'Sentret', 'Furret', 'Hoothoot', 'Noctowl', 'Ledyba', 'Ledian', 'Spinarak', 'Ariados', 'Crobat', 'Chinchou', 'Lanturn', 'Pichu', 'Cleffa', 'Igglybuff', 'Togepi', 'Togetic', 'Natu', 'Xatu', 'Mareep', 'Flaaffy', 'Ampharos', 'Ampharos Mega Ampharos', 'Bellossom', 'Marill', 'Azumarill', 'Sudowoodo', 'Politoed', 'Hoppip', 'Skiploom', 'Jumpluff', 'Aipom', 'Sunkern', 'Sunflora', 'Yanma', 'Wooper', 'Wooper Paldean Wooper', 'Quagsire', 'Espeon', 'Umbreon', 'Murkrow', 'Slowking', 'Slowking Galarian Slowking', 'Misdreavus', 'Unown', 'Wobbuffet', 'Girafarig', 'Pineco', 'Forretress', 'Dunsparce', 'Gligar', 'Steelix', 'Steelix Mega Steelix', 'Snubbull', 'Granbull', 'Qwilfish', 'Qwilfish Hisuian Qwilfish', 'Scizor', 'Scizor Mega Scizor', 'Shuckle', 'Heracross', 'Heracross Mega Heracross', 'Sneasel', 'Sneasel Hisuian Sneasel', 'Teddiursa', 'Ursaring', 'Slugma', 'Magcargo', 'Swinub', 'Piloswine', 'Corsola', 'Corsola Galarian Corsola', 'Remoraid', 'Octillery', 'Delibird', 'Mantine', 'Skarmory', 'Houndour', 'Houndoom', 'Houndoom Mega Houndoom', 'Kingdra', 'Phanpy', 'Donphan', 'Porygon2', 'Stantler', 'Smeargle', 'Tyrogue', 'Hitmontop', 'Smoochum', 'Elekid', 'Magby', 'Miltank', 'Blissey', 'Raikou', 'Entei', 'Suicune', 'Larvitar', 'Pupitar', 'Tyranitar', 'Tyranitar Mega Tyranitar', 'Lugia', 'Ho-oh', 'Celebi', 'Treecko', 'Grovyle', 'Sceptile', 'Sceptile Mega Sceptile', 'Torchic', 'Combusken', 'Blaziken', 'Blaziken Mega Blaziken', 'Mudkip', 'Marshtomp', 'Swampert', 'Swampert Mega Swampert', 'Poochyena', 'Mightyena', 'Zigzagoon', 'Zigzagoon Galarian Zigzagoon', 'Linoone', 'Linoone Galarian Linoone', 'Wurmple', 'Silcoon', 'Beautifly', 'Cascoon', 'Dustox', 'Lotad', 'Lombre', 'Ludicolo', 'Seedot', 'Nuzleaf', 'Shiftry', 'Taillow', 'Swellow', 'Wingull', 'Pelipper', 'Ralts', 'Kirlia', 'Gardevoir', 'Gardevoir Mega Gardevoir', 'Surskit', 'Masquerain', 'Shroomish', 'Breloom', 'Slakoth', 'Vigoroth', 'Slaking', 'Nincada', 'Ninjask', 'Shedinja', 'Whismur', 'Loudred', 'Exploud', 'Makuhita', 'Hariyama', 'Azurill', 'Nosepass', 'Skitty', 'Delcatty', 'Sableye', 'Sableye Mega Sableye', 'Mawile', 'Mawile Mega Mawile', 'Aron', 'Lairon', 'Aggron', 'Aggron Mega Aggron', 'Meditite', 'Medicham', 'Medicham Mega Medicham', 'Electrike', 'Manectric', 'Manectric Mega Manectric', 'Plusle', 'Minun', 'Volbeat', 'Illumise', 'Roselia', 'Gulpin', 'Swalot', 'Carvanha', 'Sharpedo', 'Sharpedo Mega Sharpedo', 'Wailmer', 'Wailord', 'Numel', 'Camerupt', 'Camerupt Mega Camerupt', 'Torkoal', 'Spoink', 'Grumpig', 'Spinda', 'Trapinch', 'Vibrava', 'Flygon', 'Cacnea', 'Cacturne', 'Swablu', 'Altaria', 'Altaria Mega Altaria', 'Zangoose', 'Seviper', 'Lunatone', 'Solrock', 'Barboach', 'Whiscash', 'Corphish', 'Crawdaunt', 'Baltoy', 'Claydol', 'Lileep', 'Cradily', 'Anorith', 'Armaldo', 'Feebas', 'Milotic', 'Castform', 'Castform Sunny Form', 'Castform Rainy Form', 'Castform Snowy Form', 'Kecleon', 'Shuppet', 'Banette', 'Banette Mega Banette', 'Duskull', 'Dusclops', 'Tropius', 'Chimecho', 'Absol', 'Absol Mega Absol', 'Wynaut', 'Snorunt', 'Glalie', 'Glalie Mega Glalie', 'Spheal', 'Sealeo', 'Walrein', 'Clamperl', 'Huntail', 'Gorebyss', 'Relicanth', 'Luvdisc', 'Bagon', 'Shelgon', 'Salamence', 'Salamence Mega Salamence', 'Beldum', 'Metang', 'Metagross', 'Metagross Mega Metagross', 'Regirock', 'Regice', 'Registeel', 'Latias', 'Latias Mega Latias', 'Latios', 'Latios Mega Latios', 'Kyogre', 'Kyogre Primal Kyogre', 'Groudon', 'Groudon Primal Groudon', 'Rayquaza', 'Rayquaza Mega Rayquaza', 'Jirachi', 'Deoxys Normal Forme', 'Deoxys Attack Forme', 'Deoxys Defense Forme', 'Deoxys Speed Forme', 'Turtwig', 'Grotle', 'Torterra', 'Chimchar', 'Monferno', 'Infernape', 'Piplup', 'Prinplup', 'Empoleon', 'Starly', 'Staravia', 'Staraptor', 'Bidoof', 'Bibarel', 'Kricketot', 'Kricketune', 'Shinx', 'Luxio', 'Luxray', 'Budew', 'Roserade', 'Cranidos', 'Rampardos', 'Shieldon', 'Bastiodon', 'Burmy Plant Cloak', 'Burmy Sandy Cloak', 'Burmy Trash Cloak', 'Wormadam Plant Cloak', 'Wormadam Sandy Cloak', 'Wormadam Trash Cloak', 'Mothim', 'Combee', 'Vespiquen', 'Pachirisu', 'Buizel', 'Floatzel', 'Cherubi', 'Cherrim', 'Shellos', 'Gastrodon', 'Ambipom', 'Drifloon', 'Drifblim', 'Buneary', 'Lopunny', 'Lopunny Mega Lopunny', 'Mismagius', 'Honchkrow', 'Glameow', 'Purugly', 'Chingling', 'Stunky', 'Skuntank', 'Bronzor', 'Bronzong', 'Bonsly', 'Mime Jr.', 'Happiny', 'Chatot', 'Spiritomb', 'Gible', 'Gabite', 'Garchomp', 'Garchomp Mega Garchomp', 'Munchlax', 'Riolu', 'Lucario', 'Lucario Mega Lucario', 'Hippopotas', 'Hippowdon', 'Skorupi', 'Drapion', 'Croagunk', 'Toxicroak', 'Carnivine', 'Finneon', 'Lumineon', 'Mantyke', 'Snover', 'Abomasnow', 'Abomasnow Mega Abomasnow', 'Weavile', 'Magnezone', 'Lickilicky', 'Rhyperior', 'Tangrowth', 'Electivire', 'Magmortar', 'Togekiss', 'Yanmega', 'Leafeon', 'Glaceon', 'Gliscor', 'Mamoswine', 'Porygon-Z', 'Gallade', 'Gallade Mega Gallade', 'Probopass', 'Dusknoir', 'Froslass', 'Rotom', 'Rotom Heat Rotom', 'Rotom Wash Rotom', 'Rotom Frost Rotom', 'Rotom Fan Rotom', 'Rotom Mow Rotom', 'Uxie', 'Mesprit', 'Azelf', 'Dialga', 'Dialga Origin Forme', 'Palkia', 'Palkia Origin Forme', 'Heatran', 'Regigigas', 'Giratina Altered Forme', 'Giratina Origin Forme', 'Cresselia', 'Phione', 'Manaphy', 'Darkrai', 'Shaymin Land Forme', 'Shaymin Sky Forme', 'Arceus', 'Victini', 'Snivy', 'Servine', 'Serperior', 'Tepig', 'Pignite', 'Emboar', 'Oshawott', 'Dewott', 'Samurott', 'Samurott Hisuian Samurott', 'Patrat', 'Watchog', 'Lillipup', 'Herdier', 'Stoutland', 'Purrloin', 'Liepard', 'Pansage', 'Simisage', 'Pansear', 'Simisear', 'Panpour', 'Simipour', 'Munna', 'Musharna', 'Pidove', 'Tranquill', 'Unfezant', 'Blitzle', 'Zebstrika', 'Roggenrola', 'Boldore', 'Gigalith', 'Woobat', 'Swoobat', 'Drilbur', 'Excadrill', 'Audino', 'Audino Mega Audino', 'Timburr', 'Gurdurr', 'Conkeldurr', 'Tympole', 'Palpitoad', 'Seismitoad', 'Throh', 'Sawk', 'Sewaddle', 'Swadloon', 'Leavanny', 'Venipede', 'Whirlipede', 'Scolipede', 'Cottonee', 'Whimsicott', 'Petilil', 'Lilligant', 'Lilligant Hisuian Lilligant', 'Basculin Red-Striped Form', 'Basculin Blue-Striped Form', 'Basculin White-Striped Form', 'Sandile', 'Krokorok', 'Krookodile', 'Darumaka', 'Darumaka Galarian Darumaka', 'Darmanitan Standard Mode', 'Darmanitan Zen Mode', 'Darmanitan Galarian Standard Mode', 'Darmanitan Galarian Zen Mode', 'Maractus', 'Dwebble', 'Crustle', 'Scraggy', 'Scrafty', 'Sigilyph', 'Yamask', 'Yamask Galarian Yamask', 'Cofagrigus', 'Tirtouga', 'Carracosta', 'Archen', 'Archeops', 'Trubbish', 'Garbodor', 'Zorua', 'Zorua Hisuian Zorua', 'Zoroark', 'Zoroark Hisuian Zoroark', 'Minccino', 'Cinccino', 'Gothita', 'Gothorita', 'Gothitelle', 'Solosis', 'Duosion', 'Reuniclus', 'Ducklett', 'Swanna', 'Vanillite', 'Vanillish', 'Vanilluxe', 'Deerling', 'Sawsbuck', 'Emolga', 'Karrablast', 'Escavalier', 'Foongus', 'Amoonguss', 'Frillish', 'Jellicent', 'Alomomola', 'Joltik', 'Galvantula', 'Ferroseed', 'Ferrothorn', 'Klink', 'Klang', 'Klinklang', 'Tynamo', 'Eelektrik', 'Eelektross', 'Elgyem', 'Beheeyem', 'Litwick', 'Lampent', 'Chandelure', 'Axew', 'Fraxure', 'Haxorus', 'Cubchoo', 'Beartic', 'Cryogonal', 'Shelmet', 'Accelgor', 'Stunfisk', 'Stunfisk Galarian Stunfisk', 'Mienfoo', 'Mienshao', 'Druddigon', 'Golett', 'Golurk', 'Pawniard', 'Bisharp', 'Bouffalant', 'Rufflet', 'Braviary', 'Braviary Hisuian Braviary', 'Vullaby', 'Mandibuzz', 'Heatmor', 'Durant', 'Deino', 'Zweilous', 'Hydreigon', 'Larvesta', 'Volcarona', 'Cobalion', 'Terrakion', 'Virizion', 'Tornadus Incarnate Forme', 'Tornadus Therian Forme', 'Thundurus Incarnate Forme', 'Thundurus Therian Forme', 'Reshiram', 'Zekrom', 'Landorus Incarnate Forme', 'Landorus Therian Forme', 'Kyurem', 'Kyurem White Kyurem', 'Kyurem Black Kyurem', 'Keldeo Ordinary Form', 'Keldeo Resolute Form', 'Meloetta Aria Forme', 'Meloetta Pirouette Forme', 'Genesect', 'Chespin', 'Quilladin', 'Chesnaught', 'Fennekin', 'Braixen', 'Delphox', 'Froakie', 'Frogadier', 'Greninja', 'Greninja Ash-Greninja', 'Bunnelby', 'Diggersby', 'Fletchling', 'Fletchinder', 'Talonflame', 'Scatterbug', 'Spewpa', 'Vivillon', 'Litleo', 'Pyroar', 'Flabébé', 'Floette', 'Florges', 'Skiddo', 'Gogoat', 'Pancham', 'Pangoro', 'Furfrou', 'Espurr', 'Meowstic Male', 'Meowstic Female', 'Honedge', 'Doublade', 'Aegislash Shield Forme', 'Aegislash Blade Forme', 'Spritzee', 'Aromatisse', 'Swirlix', 'Slurpuff', 'Inkay', 'Malamar', 'Binacle', 'Barbaracle', 'Skrelp', 'Dragalge', 'Clauncher', 'Clawitzer', 'Helioptile', 'Heliolisk', 'Tyrunt', 'Tyrantrum', 'Amaura', 'Aurorus', 'Sylveon', 'Hawlucha', 'Dedenne', 'Carbink', 'Goomy', 'Sliggoo', 'Sliggoo Hisuian Sliggoo', 'Goodra', 'Goodra Hisuian Goodra', 'Klefki', 'Phantump', 'Trevenant', 'Pumpkaboo Average Size', 'Pumpkaboo Small Size', 'Pumpkaboo Large Size', 'Pumpkaboo Super Size', 'Gourgeist Average Size', 'Gourgeist Small Size', 'Gourgeist Large Size', 'Gourgeist Super Size', 'Bergmite', 'Avalugg', 'Avalugg Hisuian Avalugg', 'Noibat', 'Noivern', 'Xerneas', 'Yveltal', 'Zygarde 50% Forme', 'Zygarde 10% Forme', 'Zygarde Complete Forme', 'Diancie', 'Diancie Mega Diancie', 'Hoopa Hoopa Confined', 'Hoopa Hoopa Unbound', 'Volcanion', 'Rowlet', 'Dartrix', 'Decidueye', 'Decidueye Hisuian Decidueye', 'Litten', 'Torracat', 'Incineroar', 'Popplio', 'Brionne', 'Primarina', 'Pikipek', 'Trumbeak', 'Toucannon', 'Yungoos', 'Gumshoos', 'Grubbin', 'Charjabug', 'Vikavolt', 'Crabrawler', 'Crabominable', 'Oricorio Baile Style', 'Oricorio Pom-Pom Style', "Oricorio Pa'u Style", 'Oricorio Sensu Style', 'Cutiefly', 'Ribombee', 'Rockruff', 'Rockruff Own Tempo Rockruff', 'Lycanroc Midday Form', 'Lycanroc Midnight Form', 'Lycanroc Dusk Form', 'Wishiwashi Solo Form', 'Wishiwashi School Form', 'Mareanie', 'Toxapex', 'Mudbray', 'Mudsdale', 'Dewpider', 'Araquanid', 'Fomantis', 'Lurantis', 'Morelull', 'Shiinotic', 'Salandit', 'Salazzle', 'Stufful', 'Bewear', 'Bounsweet', 'Steenee', 'Tsareena', 'Comfey', 'Oranguru', 'Passimian', 'Wimpod', 'Golisopod', 'Sandygast', 'Palossand', 'Pyukumuku', 'Type: Null', 'Silvally', 'Minior Meteor Form', 'Minior Core Form', 'Komala', 'Turtonator', 'Togedemaru', 'Mimikyu', 'Bruxish', 'Drampa', 'Dhelmise', 'Jangmo-o', 'Hakamo-o', 'Kommo-o', 'Tapu Koko', 'Tapu Lele', 'Tapu Bulu', 'Tapu Fini', 'Cosmog', 'Cosmoem', 'Solgaleo', 'Lunala', 'Nihilego', 'Buzzwole', 'Pheromosa', 'Xurkitree', 'Celesteela', 'Kartana', 'Guzzlord', 'Necrozma', 'Necrozma Dusk Mane Necrozma', 'Necrozma Dawn Wings Necrozma', 'Necrozma Ultra Necrozma', 'Magearna', 'Marshadow', 'Poipole', 'Naganadel', 'Stakataka', 'Blacephalon', 'Zeraora', 'Meltan', 'Melmetal', 'Grookey', 'Thwackey', 'Rillaboom', 'Scorbunny', 'Raboot', 'Cinderace', 'Sobble', 'Drizzile', 'Inteleon', 'Skwovet', 'Greedent', 'Rookidee', 'Corvisquire', 'Corviknight', 'Blipbug', 'Dottler', 'Orbeetle', 'Nickit', 'Thievul', 'Gossifleur', 'Eldegoss', 'Wooloo', 'Dubwool', 'Chewtle', 'Drednaw', 'Yamper', 'Boltund', 'Rolycoly', 'Carkol', 'Coalossal', 'Applin', 'Flapple', 'Appletun', 'Silicobra', 'Sandaconda', 'Cramorant', 'Arrokuda', 'Barraskewda', 'Toxel', 'Toxtricity Amped Form', 'Toxtricity Low Key Form', 'Sizzlipede', 'Centiskorch', 'Clobbopus', 'Grapploct', 'Sinistea', 'Polteageist', 'Hatenna', 'Hattrem', 'Hatterene', 'Impidimp', 'Morgrem', 'Grimmsnarl', 'Obstagoon', 'Perrserker', 'Cursola', "Sirfetch'd", 'Mr. Rime', 'Runerigus', 'Milcery', 'Alcremie', 'Falinks', 'Pincurchin', 'Snom', 'Frosmoth', 'Stonjourner', 'Eiscue Ice Face', 'Eiscue Noice Face', 'Indeedee Male', 'Indeedee Female', 'Morpeko Full Belly Mode', 'Morpeko Hangry Mode', 'Cufant', 'Copperajah', 'Dracozolt', 'Arctozolt', 'Dracovish', 'Arctovish', 'Duraludon', 'Dreepy', 'Drakloak', 'Dragapult', 'Zacian Hero of Many Battles', 'Zacian Crowned Sword', 'Zamazenta Hero of Many Battles', 'Zamazenta Crowned Shield', 'Eternatus', 'Eternatus Eternamax', 'Kubfu', 'Urshifu Single Strike Style', 'Urshifu Rapid Strike Style', 'Zarude', 'Regieleki', 'Regidrago', 'Glastrier', 'Spectrier', 'Calyrex', 'Calyrex Ice Rider', 'Calyrex Shadow Rider', 'Wyrdeer', 'Kleavor', 'Ursaluna', 'Ursaluna Bloodmoon', 'Basculegion Male', 'Basculegion Female', 'Sneasler', 'Overqwil', 'Enamorus Incarnate Forme', 'Enamorus Therian Forme', 'Sprigatito', 'Floragato', 'Meowscarada', 'Fuecoco', 'Crocalor', 'Skeledirge', 'Quaxly', 'Quaxwell', 'Quaquaval', 'Lechonk', 'Oinkologne Male', 'Oinkologne Female', 'Tarountula', 'Spidops', 'Nymble', 'Lokix', 'Pawmi', 'Pawmo', 'Pawmot', 'Tandemaus', 'Maushold Family of Four', 'Maushold Family of Three', 'Fidough', 'Dachsbun', 'Smoliv', 'Dolliv', 'Arboliva', 'Squawkabilly Green Plumage', 'Squawkabilly Blue Plumage', 'Squawkabilly Yellow Plumage', 'Squawkabilly White Plumage', 'Nacli', 'Naclstack', 'Garganacl', 'Charcadet', 'Armarouge', 'Ceruledge', 'Tadbulb', 'Bellibolt', 'Wattrel', 'Kilowattrel', 'Maschiff', 'Mabosstiff', 'Shroodle', 'Grafaiai', 'Bramblin', 'Brambleghast', 'Toedscool', 'Toedscruel', 'Klawf', 'Capsakid', 'Scovillain', 'Rellor', 'Rabsca', 'Flittle', 'Espathra', 'Tinkatink', 'Tinkatuff', 'Tinkaton', 'Wiglett', 'Wugtrio', 'Bombirdier', 'Finizen', 'Palafin Zero Form', 'Palafin Hero Form', 'Varoom', 'Revavroom', 'Cyclizar', 'Orthworm', 'Glimmet', 'Glimmora', 'Greavard', 'Houndstone', 'Flamigo', 'Cetoddle', 'Cetitan', 'Veluza', 'Dondozo', 'Tatsugiri Curly Form', 'Tatsugiri Droopy Form', 'Tatsugiri Stretchy Form', 'Annihilape', 'Clodsire', 'Farigiraf', 'Dudunsparce Two-Segment Form', 'Dudunsparce Three-Segment Form', 'Kingambit', 'Great Tusk', 'Scream Tail', 'Brute Bonnet', 'Flutter Mane', 'Slither Wing', 'Sandy Shocks', 'Iron Treads', 'Iron Bundle', 'Iron Hands', 'Iron Jugulis', 'Iron Moth', 'Iron Thorns', 'Frigibax', 'Arctibax', 'Baxcalibur', 'Gimmighoul Chest Form', 'Gimmighoul Roaming Form', 'Gholdengo', 'Wo-Chien', 'Chien-Pao', 'Ting-Lu', 'Chi-Yu', 'Roaring Moon', 'Iron Valiant', 'Koraidon', 'Miraidon', 'Walking Wake', 'Iron Leaves', 'Dipplin', 'Poltchageist', 'Sinistcha', 'Okidogi', 'Munkidori', 'Fezandipiti', 'Ogerpon Teal Mask', 'Ogerpon Wellspring Mask', 'Ogerpon Hearthflame Mask', 'Ogerpon Cornerstone Mask', 'Archaludon', 'Hydrapple', 'Gouging Fire', 'Raging Bolt', 'Iron Boulder', 'Iron Crown', 'Terapagos Normal Form', 'Terapagos Terastal Form', 'Terapagos Stellar Form', 'Pecharunt', "Mewtwos"],
    "CHARACTER": 
    [
        "Ash", "Ketchum", "Ash Ketchum", "Misty", "Professor Oak", "Dr. Yung", "Sir Aaron", "Lady Rin", "Tonio", "Godey", "Sheena", "Dawn", "Brock", "Dent", "Iris", "Satoshi", "Jessie", "James", "Flint", "Professor Sycamore", "Alexa", "Aila", "Jan", "Professor Burnet", "Sophocles", "Lana", "Mallow", "Kiawe", "Lillie", "Gladion", "Faba", "Lusamine", "Hala", "Olivia", "Acerola", "Kahili", "Toucannon", "Samson Oak", "Komala", "Horacio", "Pikala", "Dia", "Zeraora", "Hapu", "Ida", "Kanoa", "Tapu Fini", "Kukui", "Pisces", "Lt. Surge", "Nurse Joy"],
    "ORGANIZATION":
    [
        "Team Rocket", "Pokémon School", "Aether Foundation", "Ultra Guardians",
        "Team Skull", "Team Magma", "Team Aqua", "Team Galactic",
        "Team Plasma", "Team Flare", "Team Yell", "Team Rainbow Rocket",
        "Sword and Shield", "Elite Four", "Pokémon Rangers", "Gym Leaders", "Scientists"
    ],
    "LOCATION": 
    [
        "Cerulean City", "Pallet Town", "Vermilion City", "Alola",
        "Poni Island", "Akala Island", "Melemele Island", "Ula'ula Island",
        "Hoenn region", "Sinnoh region", "Kanto region", "Johto region",
        "Unova region", "Kalos region", "Galar region", "Lavender Town",
        "Saffron City", "Fuchsia City", "Cinnabar Island", "Pewter City",
        "Celadon City", "Goldenrod City", "Ecruteak City", "Olivine City",
        "Cianwood City", "Mahogany Town", "Blackthorn City", "Rustboro City",
        "Dewford Town", "Mauville City", "Lavaridge Town", "Fortree City",
        "Lilycove City", "Mossdeep City", "Sootopolis City", "Pacifidlog Town",
        "Ever Grande City", "Sunyshore City", "Snowpoint City", "Hearthome City",
        "Canalave City", "Floaroma Town", "Eterna City", "Celestic Town",
        "Jubilife City", "Nimbasa City", "Driftveil City", "Mistralton City",
        "Icirrus City", "Opelucid City", "Anville Town", "Accumula Town",
        "Striaton City", "Nacrene City", "Castelia City", "Virbank City",
        "Humilau City", "Santalune City", "Lumiose City", "Shalour City",
        "Coumarine City", "Laverre City", "Anistar City", "Snowbelle City",
        "Kiloude City", "Motostoke", "Turffield", "Hulbury",
        "Stow-on-Side", "Ballonlea", "Hammerlocke", "Circhester",
        "Spikemuth", "Wyndon", "Hightech Larousse City", "Camphrier Town",
    ],
    "WORK_OF_ART": 
    [
        "Pokémon Baccer World Cup", "Alolan Detective Laki", "Battle Frontier",
        "Pokémon League", "Pokémon Contest", "Pokémon Grand Festival",
        "Pokémon Showcase", "Island Challenge", "Pokémon Gym", "Pokéathlon",
        "Pokémon World Tournament", "Battle Maison", "Battle Tower",
        "Battle Dome", "Battle Factory", "Battle Pike", "Battle Palace",
        "Battle Arena", "Battle Pyramid", "Safari Zone", "Pokémon Musical",
        "Pokémon Contest Spectacular", "Battle Royal", "Battle Agency"
    ],
    "ITEM": 
    [
        "ancient artifact", "Z-Crystal", "Z-Ring", "Beast Ball", "Primarium Z",
        "Steelium Z", "Spark Stones", "Pokédex", "Rotom Dex", "Everstone"
    ],
    "ABILITY": 
    [
        "enhance a Pokémon's ability", "Aurora Veil", "Inferno Overdrive", "Hydro Vortex",
        "All-Out Pummeling", "Breakneck Blitz", "Flash Cannon", "Z-Move", "Blindness"
    ]
}

In [11]:
!python -m spacy download en -q

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
import spacy
from spacy.tokens import Span
import pandas as pd

# Load the spaCy model outside the function to avoid reloading it on each call
nlp = spacy.load("en_core_web_sm")

def process_and_create_df(df, entity_categories):
    """
    Process episode overviews to identify entities using both spaCy's built-in NER and custom entities.
    
    Args:
        df (pd.DataFrame): DataFrame containing episode overviews.
        entity_categories (dict): Dictionary containing lists of names for custom entities.
        
    Returns:
        tuple: A tuple containing the results DataFrame and an errors DataFrame (if any).
    """
    
    results = []
    error_entries = []
    
    # Helper function to create custom entity spans
    def create_custom_ents(doc, names, label):
        return [Span(doc, token.i, token.i + 1, label=label) for token in doc if token.text in names]
    
    for index, row in df.iterrows():
        try:
            doc = nlp(row['overview_eng'])
            
            # Filter out entities based on custom names
            custom_entities = []
            for label, names in entity_categories.items():
                custom_entities.extend(create_custom_ents(doc, names, label))
            
            # Remove overlapping entities
            all_ents = {ent.text: ent for ent in doc.ents if ent.text not in sum(entity_categories.values(), [])}
            all_ents.update({ent.text: ent for ent in custom_entities})
            
            # Update document entities
            doc.ents = list(all_ents.values())
            
            # Append identified entities to results
            results.extend([
                {"Episode ID": row['id'], "Entity": ent.text, "Label": ent.label_}
                for ent in doc.ents
            ])
        
        except Exception as e:
            # Append error details
            error_entries.append({"Episode Index": index, "Error": str(e)})
    
    # Create DataFrame from results
    df_results = pd.DataFrame(results)
    
    if error_entries:
        # Create DataFrame from errors if any
        df_errors = pd.DataFrame(error_entries)
        return df_results, df_errors
    return df_results, None


In [13]:
results_df, errors_df = process_and_create_df(df, entity_categories) 

In [14]:
def create_list_of_dictionaries(data):
    overview_dict = {}
    # Iterate over the data and populate the dictionary
    for overview_id, entity, label in data:
        if overview_id in overview_dict:
            overview_dict[overview_id].append((entity, label))
        else:
            overview_dict[overview_id] = [(entity, label)]
    # Convert the dictionary into a list of dictionaries with the desired structure
    list_of_dictionaries = [{overview_id: entities} for overview_id, entities in overview_dict.items()]
    return list_of_dictionaries

# Call the function with the sample data
list_of_dicts = create_list_of_dictionaries(results_df.values.tolist())
list_of_dicts

[{316019: [('Scientists', 'ORGANIZATION'),
   ('Pokémon', 'ORG'),
   ('Mewtwo', 'POKEMON')]},
 {316020: [('Ash', 'CHARACTER'),
   ('Ketchum', 'CHARACTER'),
   ('three', 'CARDINAL'),
   ('the Orange Islands', 'LOC')]},
 {208475: [('Greenfield', 'GPE'), ('Pokémon', 'ORG')]},
 {316023: [('Ash', 'CHARACTER'),
   ('Ketchum', 'CHARACTER'),
   ('Kanto', 'GPE'),
   ('Johto', 'GPE'),
   ('Pokémon', 'ORG'),
   ('Bug Pokémon', 'PERSON'),
   ('Mt. Quena', 'ORG'),
   ('Mewtwo', 'POKEMON'),
   ('Pikachu', 'POKEMON'),
   ('Meowth', 'POKEMON')]},
 {394456: [('Pokémon', 'ORG'),
   ('Forty years ago', 'DATE'),
   ('Celebi', 'POKEMON'),
   ('Trainer', 'ORG'),
   ('Sammy', 'PERSON')]},
 {362534: [('Venice', 'GPE'),
   ('Alto Mare', 'ORG'),
   ('Latias', 'POKEMON'),
   ('Pokémon Trainer', 'PERSON'),
   ('Kabutops', 'POKEMON'),
   ('the Defence Mechanism of Alto Mare', 'WORK_OF_ART'),
   ('D.M.A.', 'ORG'),
   ('Latios', 'POKEMON'),
   ('the Soul Dew', 'WORK_OF_ART')]},
 {362535: [('The Millennium Comet', 'O

In [15]:
# Ensure 'id' column exists
if 'id' in df.columns:
    # Initialize the 'labels' column with empty lists
    dfclone['labels'] = dfclone.apply(lambda x: [], axis=1)

    # Aggregate labels for each ID in a temporary dictionary
    labels_dict = {}
    for d in list_of_dicts:
        for k, v in d.items():
            if k in labels_dict:
                labels_dict[k].extend(v)
            else:
                labels_dict[k] = v

    # Update the 'labels' column in df with aggregated labels
    for index, row in df.iterrows():
        if row['id'] in labels_dict:
            dfclone.at[index, 'labels'] = labels_dict[row['id']]

dfclone.head(10)


,id,overview_eng,labels
0,316019,"Scientists genetically create a new Pokémon, M...","[(Scientists, ORGANIZATION), (Pokémon, ORG), (..."
1,316020,Ash Ketchum must gather the three spheres of f...,"[(Ash, CHARACTER), (Ketchum, CHARACTER), (thre..."
2,208475,"In the town of Greenfield, a young, lonely gir...","[(Greenfield, GPE), (Pokémon, ORG)]"
3,316023,"Mewtwo, his heart having been softened by the ...","[(Ash, CHARACTER), (Ketchum, CHARACTER), (Kant..."
4,394456,There are always those who seek to contain and...,"[(Pokémon, ORG), (Forty years ago, DATE), (Cel..."
5,362534,The film is set in the Venice-esque city of Al...,"[(Venice, GPE), (Alto Mare, ORG), (Latias, POK..."
6,362535,The Millennium Comet is about to make its long...,"[(The Millennium Comet, ORG), (Ash, CHARACTER)..."
7,362536,High-tech Larousse City is a pretty impressive...,[]
8,315865,"In the legendary past, before the creation of ...","[(Poké Balls, ORG), (two, CARDINAL), (Cameran ..."
9,394459,Charmander's voice calls out from the darkness...,"[(Charmander, POKEMON), (Pokémon, PERSON), (Sq..."


In [16]:
dfclone.to_csv('pokemon_overview_entities.csv', index=False)

In [17]:
import pandas as pd
import ast
import re

# Load the data from the CSV file
file_path = './pokemon_overview_entities.csv'  # Update this to your actual file path
df = pd.read_csv(file_path)

# Function to convert labels column from string to list
def convert_labels(label_str):
    return ast.literal_eval(label_str)

# Function to preprocess text (e.g., removing punctuation)
def preprocess_text(text):
    return re.sub(r'[^\w\s]', '', text)

# Function to tag a sentence in IOB2 format
def tag_sentence(sentence, labels):
    labels = convert_labels(labels)
    sentence = preprocess_text(sentence)
    tokens = sentence.split()
    tags = ['O'] * len(tokens)

    for label, label_type in labels:
        label_tokens = preprocess_text(label).split()
        for i in range(len(tokens)):
            if tokens[i:i+len(label_tokens)] == label_tokens:
                tags[i] = f"B-{label_type}"
                for j in range(1, len(label_tokens)):
                    tags[i+j] = f"I-{label_type}"

    return list(zip(tokens, tags))

# Apply the tagging function to each row in the DataFrame
def process_row(row):
    return tag_sentence(row['overview_eng'], row['labels'])

# Assuming the DataFrame `df` has the columns 'overview_eng' and 'labels'
tagged_sentences = df.apply(process_row, axis=1)

# If you want to print the tagged sentences for the first few rows, for example
for index, tagged_sentence in tagged_sentences.iteritems():
    print(f"Sentence {index+1}:")
    for word, tag in tagged_sentence:
        print(f"{word} {tag}")
    print("\n")


FileNotFoundError: [Errno 2] No such file or directory: 'E:/NLP/pokemon_overview_entities.csv'